# NLP: Trabajo Práctico N°1
**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : 

# Importamos librerías necesarias

In [1]:
from rqst import *

C:\Users\julil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Carga de datasets

In [2]:
dataset_juegos = pd.read_csv('./src/bgg_database.csv')
dataset_peliculas = pd.read_csv('./src/IMDB-Movie-Data.csv')
dataset_libros = pd.read_csv('./src/dataset_libros.csv')

## Traducciones de Datasets

# Análisis Datasets

En este apartado se analiza la presencia de null, distribuciones de los datasets, balanceo, etc.

## Dataset Juegos

## Dataset Películas

## Dataset Libros